In [23]:
# Cuts used to go from o.summary.fits to o.gst.fits

#snr = 5.0
#sharp = 0.04
#crowd = 0.5
#objtype = 1
#flag = 99

from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u


In [7]:
path='test_photometry_output/'

In [17]:
photometry=fits.open(path+'o.gst.fits')[1]

In [30]:
tbl=Table(photometry.data)
tbl.colnames

['chip',
 'RA',
 'DEC',
 'X',
 'Y',
 'OBJECT_TYPE',
 'F555W_VEGA',
 'F555W_ERR',
 'F555W_SNR',
 'F555W_SHARP',
 'F555W_ROUND',
 'F555W_CROWD',
 'F555W_FLAG']

In [24]:
sn2021sjt_position = SkyCoord(ra=309.33000691667*u.degree, dec=+66.106417016667*u.degree)


In [25]:
sn2021sjt_position

<SkyCoord (ICRS): (ra, dec) in deg
    (309.33000692, 66.10641702)>

In [36]:
catalog = SkyCoord(ra=tbl['RA']*u.degree, dec=tbl['DEC']*u.degree)

In [49]:
# Now idx are indices into catalog that are the closest objects to each of the coordinates in c
# d2d are the on-sky distances between them,
# d3d are the 3-dimensional distances.
# Because coordinate objects support indexing, idx enables easy access to the matched set of coordinates 
# in the catalog:

idx, d2d, d3d = sn2021sjt_position.match_to_catalog_sky(catalog)

In [47]:
max_sep = 30.0 * u.arcsec


In [50]:
d2d

<Angle [1.50277683e-05] deg>

In [51]:
tbl['RA']

309.30106986096405
309.38514594955814
309.31574638711186
309.3271923382695
309.3346308089143
309.3205936192496
309.3929272557338
309.3169477501365
309.2958953010367
309.39314082828946
309.36550235238764
